# User comparison tests

# Table of Contents

[Preparation](#preparation)

[User data vectors](#userdatavectors)

[User lists](#userlists)

[Sessions' checkpoints](#sessionscheckpoints)

[Assembly](#assembly)

[Time](#time)

# Preparation
<a id=preparation />

In [ ]:
%run "../Functions/1. Google form analysis.ipynb"
%run "../Functions/4. User comparison.ipynb"

# Data vectors of users
<a id=userdatavectors />

In [ ]:
#getAllResponders()

In [ ]:
setAnswerTemporalities(gform)

In [ ]:
# small sample
#allData = getAllUserVectorData( getAllUsers( df152 )[:10] )

# complete set
#allData = getAllUserVectorData( getAllUsers( df152 ) )

# subjects who answered the gform
allData = getAllUserVectorData( getAllResponders() )

# 10 subjects who answered the gform
#allData = getAllUserVectorData( getAllResponders()[:10] )

In [ ]:
gform['Temporality'].unique()

In [ ]:
allData.loc['scoreundefined'].dropna()

In [ ]:
getAllUsers(df152)[:10]

In [ ]:
len(getAllUsers(df152))

# List of users and their sessions
<a id=userlists />

In [ ]:
userSessionsRelevantColumns = ['customData.localplayerguid', 'sessionId']
userSessions = df152[df152['type']=='start'].loc[:,userSessionsRelevantColumns]

In [ ]:
userSessions = userSessions.rename(index=str, columns={'customData.localplayerguid': 'userId'})
userSessions.head()

In [ ]:
#groupedUserSessions = userSessions.groupby('customData.localplayerguid')
#groupedUserSessions.head()
#groupedUserSessions.describe().head()

# List of sessions with their checkpoints achievements
<a id=sessionscheckpoints />

In [ ]:
checkpointsRelevantColumns = ['sessionId', 'customData.localplayerguid', 'type', 'section', 'userTime']
checkpoints = df152.loc[:, checkpointsRelevantColumns]

checkpoints = checkpoints[checkpoints['type']=='reach'].loc[:,['section','sessionId','userTime']]
checkpoints = checkpoints[checkpoints['section'].str.startswith('tutorial', na=False)]
#checkpoints = checkpoints.groupby("sessionId")
#checkpoints = checkpoints.max()
checkpoints.head()

# Assembly of both
<a id=assembly />

In [ ]:
#assembled = userSessions.combine_first(checkpoints)
assembled = pd.merge(userSessions, checkpoints, on='sessionId', how='outer')
assembled.head()

In [ ]:
userSections = assembled.drop('sessionId', 1)
userSections.head()

In [ ]:
userSections = userSections.dropna()
userSections.head()

In [ ]:
checkpoints = userSections.groupby("userId")
checkpoints = checkpoints.max()
checkpoints.head()

# Time analysis
<a id=time />

In [ ]:
#userTimedSections = userSections.groupby("userId").agg({ "userTime": np.min })
#userTimedSections = userSections.groupby("userId")
userTimes = userSections.groupby("userId").agg({ "userTime": [np.min, np.max] })
userTimes["duration"] = pd.to_datetime(userTimes["userTime"]["amax"]) - pd.to_datetime(userTimes["userTime"]["amin"])
userTimes["duration"] = userTimes["duration"].map(lambda x: np.timedelta64(x, 's'))
userTimes = userTimes.sort_values(by=['duration'], ascending=[False])
userTimes.head()

In [ ]:
userTimes.loc[:,'duration']
userTimes = userTimes[4:]
userTimes["duration_seconds"] = userTimes["duration"].map(lambda x: pd.Timedelta(x).seconds)
maxDuration = np.max(userTimes["duration_seconds"])
userTimes["duration_rank"] = userTimes["duration_seconds"].rank(ascending=False)
userTimes.plot(x="duration_rank", y="duration_seconds")
plt.xlabel("game session")
plt.ylabel("time played (s)")
plt.legend('')
plt.xlim(0, 139)
plt.ylim(0, maxDuration)

In [ ]:
userTimedSections = userSections.groupby("section").agg({ "userTime": np.min })
userTimedSections

In [ ]:
userTimedSections["firstReached"] = pd.to_datetime(userTimedSections["userTime"])
userTimedSections.head()

In [ ]:
userTimedSections.drop('userTime', 1)
userTimedSections.head()

In [ ]:
userTimedSections["firstCompletionDuration"] = userTimedSections["firstReached"].diff()
userTimedSections.head()